In [50]:
# Imports from Jonathan's wkbk

import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np

In [51]:
# Load env data
load_dotenv()

True

In [52]:
# Assign Alpaca keys to variables
alpaca_api_key = os.getenv("APCA_API_KEY_ID")
alpaca_secret_key = os.getenv("APCA_API_SECRET_KEY")

In [53]:
# Initialize Alpaca connection
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2'
)

In [54]:
# Set start and end dates for
start_date = pd.Timestamp('2020-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2020-12-31', tz='America/New_York').isoformat()

In [55]:
# Assign default ticker symbol
ticker_string = 'JPM'
ticker = [ticker_string]

In [56]:
# Assign default time frame
timeframe = '15Min'

In [57]:
# Get ticker historical data
df_ohlc_original = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date
).df

In [58]:
# Copy of original data
df_ohlc = df_ohlc_original

# Verify data in dataframe
df_ohlc = df_ohlc[ticker_string]
df_ohlc.head()

,open,high,low,close,volume
2020-01-02 09:30:00-05:00,140.070,140.14,139.52,139.670,29953
2020-01-02 09:45:00-05:00,139.675,139.91,139.44,139.670,24343
2020-01-02 10:00:00-05:00,139.635,139.81,139.52,139.640,19021
2020-01-02 10:15:00-05:00,139.600,139.60,139.34,139.400,18856
2020-01-02 10:30:00-05:00,139.460,139.54,139.28,139.535,19695


In [59]:
# Window size
bollinger_band_window = 20
num_standard_deviation = 2

# Calculate mean and standard deviation
df_ohlc['bollinger_band_middle'] = df_ohlc['close'].rolling(window=bollinger_band_window).mean()
df_ohlc['bollinger_band_std'] = df_ohlc['close'].rolling(window=bollinger_band_window).std()

# Calculate upper bollinger band and lower bollinger band
df_ohlc['bollinger_band_upper'] = df_ohlc['bollinger_band_middle'] + (df_ohlc['bollinger_band_std'] * num_standard_deviation)
df_ohlc['bollinger_band_lower'] = df_ohlc['bollinger_band_middle'] - (df_ohlc['bollinger_band_std'] * num_standard_deviation)

df_ohlc.head()

,open,high,low,close,volume,bollinger_band_middle,bollinger_band_std,bollinger_band_upper,bollinger_band_lower
2020-01-02 09:30:00-05:00,140.070,140.14,139.52,139.670,29953,NaN,NaN,NaN,NaN
2020-01-02 09:45:00-05:00,139.675,139.91,139.44,139.670,24343,NaN,NaN,NaN,NaN
2020-01-02 10:00:00-05:00,139.635,139.81,139.52,139.640,19021,NaN,NaN,NaN,NaN
2020-01-02 10:15:00-05:00,139.600,139.60,139.34,139.400,18856,NaN,NaN,NaN,NaN
2020-01-02 10:30:00-05:00,139.460,139.54,139.28,139.535,19695,NaN,NaN,NaN,NaN


In [60]:
# Create signal for long
df_ohlc['bollinger_band_long'] = np.where(df_ohlc['close'] > df_ohlc['bollinger_band_upper'], 1.0, 0.0)
df_ohlc.loc[df_ohlc['bollinger_band_long'] == 1.0].head()

,open,high,low,close,volume,bollinger_band_middle,bollinger_band_std,bollinger_band_upper,bollinger_band_lower,bollinger_band_long
2020-01-06 15:45:00-05:00,137.810,138.270,137.800,138.210,49640,137.65825,0.185304,138.028858,137.287642,1.0
2020-01-08 11:30:00-05:00,136.720,137.135,136.420,137.135,21206,136.42250,0.318357,137.059214,135.785786,1.0
2020-01-08 11:45:00-05:00,137.200,137.400,137.070,137.240,13245,136.48500,0.350098,137.185195,135.784805,1.0
2020-01-13 13:00:00-05:00,136.315,136.480,136.295,136.460,6740,136.06925,0.178990,136.427231,135.711269,1.0
2020-01-13 15:45:00-05:00,136.950,137.170,136.880,137.170,57073,136.44800,0.357095,137.162190,135.733810,1.0


In [61]:
df_ohlc.dropna(inplace=True)
df_ohlc.isna().sum()

open                     0
high                     0
low                      0
close                    0
volume                   0
bollinger_band_middle    0
bollinger_band_std       0
bollinger_band_upper     0
bollinger_band_lower     0
bollinger_band_long      0
dtype: int64

In [62]:
# Bollinger Band function to capture signal
def bollinger_band_signal_generator(dataframe_name, closing_price_column_name = 'close', bollinger_band_window = 20, num_standard_deviation = 2):
    """Creates Bollinger Band indicator with signal for long position based on closing price
    Args:
        dataframe_name (dict): Single security dataframe containing at least closing prices
        closing_price_column_name (str): Name of column in dataframe containing closing prices
        bollinger_band_window (int): Desired timeframe window used for rolling calculations
        num_standard_deviation (int): Desired number of standard deviations to calculate
    Returns:
        A dataframe of:
            original data passed to function,
            bollinger_band_middle (flt): Column of values for middle band,
            bollinger_band_std (flt): Column of values to calculate standard deviation,
            bollinger_band_upper (flt): Column of values for upper band,
            bollinger_band_lower (flt): Column of values for lower band,
            bollinger_band_long (flt): Column of generated signals (1.0 = True, 0.0 = False)
            """
    # Calculate mean and standard deviation
    dataframe_name['bollinger_band_middle'] = dataframe_name[closing_price_column_name].rolling(window=bollinger_band_window).mean()
    dataframe_name['bollinger_band_std'] = dataframe_name[closing_price_column_name].rolling(window=bollinger_band_window).std()

    # Calculate upper bollinger band and lower bollinger band
    dataframe_name['bollinger_band_upper'] = dataframe_name['bollinger_band_middle'] + (dataframe_name['bollinger_band_std'] * num_standard_deviation)
    dataframe_name['bollinger_band_lower'] = dataframe_name['bollinger_band_middle'] - (dataframe_name['bollinger_band_std'] * num_standard_deviation)

    # Create signal column
    dataframe_name['bollinger_band_long'] = np.where(dataframe_name[closing_price_column_name] > dataframe_name['bollinger_band_upper'], 1.0, 0.0)
    
    # Drop NaN values
    dataframe_name.dropna(inplace=True)

    # Return dataframe with features and target
    return dataframe_name

In [63]:
# Test bollinger band function
testing_df = df_ohlc_original['JPM']
new_boll_df = bollinger_band_signal_generator(testing_df, 'close')
new_boll_df[:30]

,open,high,low,close,volume,bollinger_band_middle,bollinger_band_std,bollinger_band_upper,bollinger_band_lower,bollinger_band_long
2020-01-02 14:15:00-05:00,140.575,140.770,140.525,140.730,20048,140.19825,0.420517,141.039285,139.357215,0.0
2020-01-02 14:30:00-05:00,140.745,140.870,140.690,140.870,13698,140.25825,0.426742,141.111734,139.404766,0.0
2020-01-02 14:45:00-05:00,140.845,140.980,140.800,140.980,22353,140.32375,0.432200,141.188150,139.459350,0.0
2020-01-02 15:00:00-05:00,140.960,140.985,140.865,140.870,70068,140.38525,0.417030,141.219311,139.551189,0.0
2020-01-02 15:15:00-05:00,140.865,140.890,140.780,140.820,75963,140.45625,0.357023,141.170296,139.742204,0.0
2020-01-02 15:30:00-05:00,140.815,140.910,140.775,140.840,20290,140.52150,0.293370,141.108240,139.934760,0.0
2020-01-02 15:45:00-05:00,140.820,141.060,140.790,141.060,144672,140.57850,0.280831,141.140162,140.016838,0.0
2020-01-03 09:30:00-05:00,137.340,138.370,137.300,138.055,59857,140.47825,0.623938,141.726125,139.230375,0.0
2020-01-03 09:45:00-05:00,138.080,138.330,137.970,138.210,30184,140.38200,0.802579,141.987157,138.776843,0.0
2020-01-03 10:00:00-05:00,138.100,138.100,137.590,138.015,11921,140.26925,0.961753,142.192756,138.345744,0.0


In [76]:
# Metrics/Indicators
sharpe_ratio = (new_boll_df.mean()*252) / (new_boll_df.std() * np.sqrt(252))
sharpe_ratio

open                     108.260284
high                     108.868969
low                      107.668265
close                    108.248532
volume                     8.663646
bollinger_band_middle    108.259304
bollinger_band_std        17.875607
bollinger_band_upper     111.488936
bollinger_band_lower     104.310552
bollinger_band_long        4.139105
dtype: float64

In [77]:
# Historical Price Vol

# calc daily logarithmic return
new_boll_df['returns'] = (np.log(new_boll_df.close / new_boll_df.close.shift(-1)))

# calc daily std of returns
daily_std = np.std(new_boll_df.returns)

# annualized daily std
std = daily_std * 252 ** 0.5

In [3]:
# Implied Vol

In [5]:
# Sortino Ratio